In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

df = pd.read_csv('usa_00050.csv.gz')

print(df)

      YEAR  SAMPLE   SERIAL       CBSERIAL   HHWT        CLUSTER  STATEFIP  \
0     2010  201001     7876         488424   90.0  2010000078761         1   
1     2010  201001     7876         488424   90.0  2010000078761         1   
2     2010  201001     7876         488424   90.0  2010000078761         1   
3     2010  201001    22464        1388149  107.0  2010000224641         1   
4     2010  201001    22464        1388149  107.0  2010000224641         1   
...    ...     ...      ...            ...    ...            ...       ...   
7033  2019  201901  1403102  2019000128243   37.0  2019014031021        55   
7034  2019  201901  1404137  2019000186916   46.0  2019014041371        55   
7035  2019  201901  1405932  2019000293128  123.0  2019014059321        55   
7036  2019  201901  1413819  2019000743427  101.0  2019014138191        55   
7037  2019  201901  1418721  2019001023584  143.0  2019014187211        55   

       PUMA  METRO   STRATA  ...  EDUC  EDUCD  EMPSTAT  EMPSTAT

In [2]:
# Generate wkswork2_0
df['wkswork2_0'] = np.nan
df.loc[df['WKSWORK2'] == 1, 'wkswork2_0'] = 7
df.loc[df['WKSWORK2'] == 2, 'wkswork2_0'] = 20
df.loc[df['WKSWORK2'] == 3, 'wkswork2_0'] = 33
df.loc[df['WKSWORK2'] == 4, 'wkswork2_0'] = 43.5
df.loc[df['WKSWORK2'] == 5, 'wkswork2_0'] = 48.5
df.loc[df['WKSWORK2'] == 6, 'wkswork2_0'] = 51

In [3]:
# Generate uhrswork0
df['uhrswork0'] = df['UHRSWORK']
df.loc[df['UHRSWORK'] == 0, 'uhrswork0'] = np.nan

In [4]:
# Generate incwage0
df['incwage0'] = df['INCWAGE']
df.loc[df['INCWAGE'] == 999999, 'incwage0'] = np.nan
df.loc[df['INCWAGE'] == 999998, 'incwage0'] = np.nan

# Generate inctot0
df['inctot0'] = df['INCTOT']
df.loc[df['INCTOT'] == 999999, 'inctot0'] = np.nan
df.loc[df['INCTOT'] == 999998, 'inctot0'] = np.nan

# Handle NaN and infinity values
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Handle invalid wage values (e.g., division by zero)
df['wage'] = df['incwage0'] / df['wkswork2_0'] / df['uhrswork0']
df['wage'] = df['wage'].replace([np.inf, -np.inf], np.nan)  # Replace infinity values with NaN


In [7]:
# Generate logwage
df['logwage'] = np.log(df['wage'])

print(df)

      YEAR  SAMPLE   SERIAL       CBSERIAL   HHWT        CLUSTER  STATEFIP  \
0     2010  201001     7876         488424   90.0  2010000078761         1   
1     2010  201001     7876         488424   90.0  2010000078761         1   
2     2010  201001     7876         488424   90.0  2010000078761         1   
3     2010  201001    22464        1388149  107.0  2010000224641         1   
4     2010  201001    22464        1388149  107.0  2010000224641         1   
...    ...     ...      ...            ...    ...            ...       ...   
7033  2019  201901  1403102  2019000128243   37.0  2019014031021        55   
7034  2019  201901  1404137  2019000186916   46.0  2019014041371        55   
7035  2019  201901  1405932  2019000293128  123.0  2019014059321        55   
7036  2019  201901  1413819  2019000743427  101.0  2019014138191        55   
7037  2019  201901  1418721  2019001023584  143.0  2019014187211        55   

       PUMA  METRO   STRATA  ...  WKSWORK2  UHRSWORK   INCTOT  

/srv/conda/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [8]:
# Handle NaN and infinity values in logwage
df['logwage'].replace([np.inf, -np.inf], np.nan, inplace=True)

/tmp/ipykernel_174/579942814.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['logwage'].replace([np.inf, -np.inf], np.nan, inplace=True)


In [ ]:
# Generate logincwage and loginctot
df['logincwage'] = np.log(df['incwage0'])
df['loginctot'] = np.log(df['inctot0'])


In [9]:
#X = df[['race_chinese', 'race_indian', 'race_other']]
#X = sm.add_constant(X)  
#y = df['logwage']

model = sm.OLS(df['logwage'], 
               sm.add_constant(df[['race_chinese', 'race_indian', 'race_other']]), 
               missing='drop').fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                logwage   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     15.30
Date:                Sun, 17 Mar 2024   Prob (F-statistic):           6.65e-10
Time:                        19:16:03   Log-Likelihood:                -5795.3
No. Observations:                4343   AIC:                         1.160e+04
Df Residuals:                    4339   BIC:                         1.162e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            3.0591      0.033     94.113   

In [23]:
# Generate wkswork2_0
df['wkswork2_0'] = np.nan
df.loc[df['wkswork2'] == 1, 'wkswork2_0'] = 7
df.loc[df['wkswork2'] == 2, 'wkswork2_0'] = 20
df.loc[df['wkswork2'] == 3, 'wkswork2_0'] = 33
df.loc[df['wkswork2'] == 4, 'wkswork2_0'] = 43.5
df.loc[df['wkswork2'] == 5, 'wkswork2_0'] = 48.5
df.loc[df['wkswork2'] == 6, 'wkswork2_0'] = 51

# Generate uhrswork0
df['uhrswork0'] = df['uhrswork']
df.loc[df['uhrswork'] == 0, 'uhrswork0'] = np.nan

# Generate incwage0
df['incwage0'] = df['incwage']
df.loc[df['incwage'] == 999999, 'incwage0'] = np.nan
df.loc[df['incwage'] == 999998, 'incwage0'] = np.nan

# Generate inctot0
df['inctot0'] = df['inctot']
df.loc[df['inctot'] == 999999, 'inctot0'] = np.nan
df.loc[df['inctot'] == 999998, 'inctot0'] = np.nan

# Handle NaN and infinity values
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Handle invalid wage values (e.g., division by zero)
df['wage'] = df['incwage0'] / df['wkswork2_0'] / df['uhrswork']
df['wage'] = df['wage'].replace([np.inf, -np.inf], np.nan)  # Replace infinity values with NaN

# Generate logwage
df['logwage'] = np.log(df['wage'])

# Handle NaN and infinity values in logwage
df['logwage'].replace([np.inf, -np.inf], np.nan, inplace=True)

# Generate logincwage and loginctot
df['logincwage'] = np.log(df['incwage0'])
df['loginctot'] = np.log(df['inctot0'])


# Now you can proceed with further analysis using the 'logwage' column

/tmp/ipykernel_119/3454163403.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['wage'] = df['wage'].replace([np.inf, -np.inf], np.nan)  # Replace infinity values with NaN
/tmp/ipykernel_119/3454163403.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['logwage'].replace([np.inf, -np.inf]

In [15]:
print(df['logwage'])

KeyError: 'logwage'

In [3]:
category_raced= df['RACED'].value_counts()
print(category_raced)

RACED
400    3523
667    1291
610     511
676     361
100     297
       ... 
642       1
922       1
854       1
920       1
833       1
Name: count, Length: 61, dtype: int64


In [4]:
category_raced.head(10)

RACED
400    3523
667    1291
610     511
676     361
100     297
665     268
640     182
671      90
600      65
620      39
Name: count, dtype: int64

In [5]:
df['RACED'] = df['RACED'].astype(str)  # Convert to string in case it's not already

# Convert numerical codes to categorical values (you may need a mapping dictionary)
# Replace 'mapping_dict' with the actual mapping dictionary for numerical codes to categories
mapping_dict = {'400': 'Chinese', '610': 'Indian','667': 'Malay'}
df['RACED_NEW'] = df['RACED'].map(mapping_dict).fillna('Other')

print(df['RACED_NEW'])

0         Other
1         Other
2         Other
3         Malay
4         Malay
         ...   
7033    Chinese
7034      Malay
7035      Other
7036      Other
7037      Malay
Name: RACED_NEW, Length: 7038, dtype: object


In [6]:
df['RACED_NEW'] = df['RACED_NEW'].astype(str)

desired_reference_category = 'Malay'

# Move the desired reference category to the front of the unique values
unique_values = df['RACED_NEW'].unique()
new_order = [desired_reference_category] + [value for value in unique_values if value != desired_reference_category]

# Update the DataFrame with the reordered category
df['RACED_NEW'] = pd.Categorical(df['RACED_NEW'], categories=new_order, ordered=True)

# Create dummy variables with drop_first=True
df_dummies= pd.get_dummies(df['RACED_NEW'], prefix='RACE', drop_first=True)

df_dummies['RACE_Other'] = df_dummies['RACE_Other'].astype(int)
df_dummies['RACE_Indian'] = df_dummies['RACE_Indian'].astype(int)
df_dummies['RACE_Chinese'] = df_dummies['RACE_Chinese'].astype(int)

df = pd.concat([df, df_dummies], axis=1)

print(df)

      YEAR  SAMPLE   SERIAL       CBSERIAL   HHWT        CLUSTER   STRATA  GQ  \
0     2010  201001     7876         488424   90.0  2010000078761   240001   1   
1     2010  201001     7876         488424   90.0  2010000078761   240001   1   
2     2010  201001     7876         488424   90.0  2010000078761   240001   1   
3     2010  201001    22464        1388149  107.0  2010000224641   130001   1   
4     2010  201001    22464        1388149  107.0  2010000224641   130001   1   
...    ...     ...      ...            ...    ...            ...      ...  ..   
7033  2019  201901  1403102  2019000128243   37.0  2019014031021   130155   1   
7034  2019  201901  1404137  2019000186916   46.0  2019014041371   160055   1   
7035  2019  201901  1405932  2019000293128  123.0  2019014059321    90055   1   
7036  2019  201901  1413819  2019000743427  101.0  2019014138191  4100555   1   
7037  2019  201901  1418721  2019001023584  143.0  2019014187211    10255   1   

      PERNUM  PERWT  ...  L

In [7]:
X = df[['RACE_Chinese', 'RACE_Indian', 'RACE_Other']]
X = sm.add_constant(X)  
y = df['INCTOT']

model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 INCTOT   R-squared:                       0.130
Model:                            OLS   Adj. R-squared:                  0.130
Method:                 Least Squares   F-statistic:                     351.3
Date:                Sat, 09 Mar 2024   Prob (F-statistic):          1.38e-212
Time:                        07:44:43   Log-Likelihood:            -1.1369e+05
No. Observations:                7038   AIC:                         2.274e+05
Df Residuals:                    7034   BIC:                         2.274e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         5.218e+05   6.98e+04      7.477   

In [8]:
import numpy as np

df['log_inctot'] = np.log(df['INCTOT'] + 1e-10)
print(df['log_inctot'])


0       11.050890
1      -23.025851
2        4.094345
3        9.472705
4        7.600902
          ...    
7033    10.463103
7034    11.066638
7035   -23.025851
7036    16.118096
7037    11.302204
Name: log_inctot, Length: 7038, dtype: float64


/srv/conda/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [9]:
print(df['INCTOT'])

0         63000
1             0
2            60
3         13000
4          2000
         ...   
7033      35000
7034      64000
7035          0
7036    9999999
7037      81000
Name: INCTOT, Length: 7038, dtype: int64


In [50]:
df['inctot0'] = df['INCTOT'].replace(999999, pd.NA)
df['inctot0'] = df['INCTOT'].replace(999998, pd.NA)
df['loginctot0'] = np.log(df['inctot0'])
df['loginctot0'] = df['loginctot0'].replace(-np.inf, pd.NA)
print(df['loginctot0'])

0        11.05089
1            <NA>
2        4.094345
3        9.472705
4        7.600902
          ...    
7033    10.463103
7034    11.066638
7035         <NA>
7036    16.118096
7037    11.302204
Name: loginctot0, Length: 7032, dtype: object


In [53]:
df2 = df.dropna(subset=['loginctot0', 'log_inctot', 'RACE_Chinese', 'RACE_Indian', 'RACE_Other'])

In [57]:
X = df2[['RACE_Chinese', 'RACE_Indian', 'RACE_Other']]
X = sm.add_constant(X)  
y = df2['loginctot0']

model = sm.OLS(y, X, missing='drop').fit()

print(model.summary())

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data). The types seen wereNone and const           float64
RACE_Chinese      int64
RACE_Indian       int64
RACE_Other        int64
dtype: object. The data was
0        11.05089
2        4.094345
3        9.472705
4        7.600902
5       11.759786
          ...    
7032    16.118096
7033    10.463103
7034    11.066638
7036    16.118096
7037    11.302204
Name: loginctot0, Length: 5861, dtype: object
and
       const  RACE_Chinese  RACE_Indian  RACE_Other
0       1.0             0            0           1
2       1.0             0            0           1
3       1.0             0            0           0
4       1.0             0            0           0
5       1.0             0            1           0
...     ...           ...          ...         ...
7032    1.0             0            0           1
7033    1.0             1            0           0
7034    1.0             0            0       

In [52]:
X = df[['RACE_Chinese', 'RACE_Indian', 'RACE_Other']]
X = sm.add_constant(X)  
y = df['log_inctot']

model = sm.OLS(y, X, missing='drop').fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             log_inctot   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     19.70
Date:                Sat, 09 Mar 2024   Prob (F-statistic):           1.03e-12
Time:                        08:20:20   Log-Likelihood:                -27881.
No. Observations:                7032   AIC:                         5.577e+04
Df Residuals:                    7028   BIC:                         5.580e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            3.7680      0.355     10.611   

In [11]:
# Check for missing values in the dataframe
print(df[['log_inctot', 'RACE_Chinese', 'RACE_Indian', 'RACE_Other']].isnull().sum())

log_inctot      6
RACE_Chinese    0
RACE_Indian     0
RACE_Other      0
dtype: int64


In [12]:
# Check for infinite values in the dataframe
print(np.isinf(df[['log_inctot', 'RACE_Chinese', 'RACE_Indian', 'RACE_Other']]).sum())


log_inctot      0
RACE_Chinese    0
RACE_Indian     0
RACE_Other      0
dtype: int64


In [13]:
df = df.dropna(subset=['INCTOT', 'log_inctot', 'RACE_Chinese', 'RACE_Indian', 'RACE_Other'])

In [14]:
print(df['SEX'])

0       2
1       1
2       1
3       1
4       2
       ..
7033    1
7034    1
7035    2
7036    2
7037    1
Name: SEX, Length: 7032, dtype: int64


In [15]:
df['SEX_2'] = df['SEX'].replace({1: 'Male', 2: 'Female'})
print(df['SEX_2'])

0       Female
1         Male
2         Male
3         Male
4       Female
         ...  
7033      Male
7034      Male
7035    Female
7036    Female
7037      Male
Name: SEX_2, Length: 7032, dtype: object


In [17]:
# Assuming 'SEX' is the column representing gender in your DataFrame
df=pd.get_dummies(df, columns=['SEX_2'], drop_first=True)
df['SEX_2_Male'] = df['SEX_2_Male'].astype(int)
print(df)

      YEAR  SAMPLE   SERIAL       CBSERIAL   HHWT        CLUSTER   STRATA  GQ  \
0     2010  201001     7876         488424   90.0  2010000078761   240001   1   
1     2010  201001     7876         488424   90.0  2010000078761   240001   1   
2     2010  201001     7876         488424   90.0  2010000078761   240001   1   
3     2010  201001    22464        1388149  107.0  2010000224641   130001   1   
4     2010  201001    22464        1388149  107.0  2010000224641   130001   1   
...    ...     ...      ...            ...    ...            ...      ...  ..   
7033  2019  201901  1403102  2019000128243   37.0  2019014031021   130155   1   
7034  2019  201901  1404137  2019000186916   46.0  2019014041371   160055   1   
7035  2019  201901  1405932  2019000293128  123.0  2019014059321    90055   1   
7036  2019  201901  1413819  2019000743427  101.0  2019014138191  4100555   1   
7037  2019  201901  1418721  2019001023584  143.0  2019014187211    10255   1   

      PERNUM  PERWT  ...   

In [18]:
X = df[['RACE_Chinese', 'RACE_Indian', 'RACE_Other','AGE','SEX_2_Male']]
#X = df[['RACE_Chinese', 'RACE_Indian', 'RACE_Other','AGE']]
X = sm.add_constant(X)  
y = df['log_inctot']

model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             log_inctot   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     42.09
Date:                Sat, 09 Mar 2024   Prob (F-statistic):           7.43e-43
Time:                        07:48:52   Log-Likelihood:                -27807.
No. Observations:                7032   AIC:                         5.563e+04
Df Residuals:                    7026   BIC:                         5.567e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            2.1254      0.512      4.152   

In [34]:
# Assuming df is your DataFrame and 'EDUCD' is the column you want to recategorize
df['yrsed'] = 0  # Initialize 'yrsed' column to 0

# Recategorize based on Stata code
df.loc[df['EDUCD'] == 0, 'yrsed'] = 0
df.loc[df['EDUCD'] == 1, 'yrsed'] = 0
df.loc[df['EDUCD'] == 2, 'yrsed'] = 0
df.loc[df['EDUCD'] == 10, 'yrsed'] = 2  # pre-2008
df.loc[df['EDUCD'] == 11, 'yrsed'] = 0
df.loc[df['EDUCD'] == 12, 'yrsed'] = 0
df.loc[df['EDUCD'] == 14, 'yrsed'] = 1
df.loc[df['EDUCD'] == 15, 'yrsed'] = 2
df.loc[df['EDUCD'] == 16, 'yrsed'] = 3
df.loc[df['EDUCD'] == 17, 'yrsed'] = 4
df.loc[df['EDUCD'] == 21, 'yrsed'] = 5.5  # pre-2008
df.loc[df['EDUCD'] == 22, 'yrsed'] = 5
df.loc[df['EDUCD'] == 23, 'yrsed'] = 6
df.loc[df['EDUCD'] == 24, 'yrsed'] = 7.5  # pre-2008
df.loc[df['EDUCD'] == 25, 'yrsed'] = 7
df.loc[df['EDUCD'] == 26, 'yrsed'] = 8
# Switch to educ
df.loc[df['EDUC'] == 3, 'yrsed'] = 9
df.loc[df['EDUC'] == 4, 'yrsed'] = 10
df.loc[df['EDUC'] == 5, 'yrsed'] = 11
# Switch to educd
df.loc[df['EDUCD'] == 61, 'yrsed'] = 11  # HS no diploma
df.loc[(df['EDUCD'] == 62) | (df['EDUCD'] == 63) | (df['EDUCD'] == 64), 'yrsed'] = 12
df.loc[df['EDUCD'] == 65, 'yrsed'] = 12.5
# Switch to educ
df.loc[df['EDUC'] == 7, 'yrsed'] = 13
df.loc[df['EDUC'] == 8, 'yrsed'] = 14
df.loc[df['EDUC'] == 9, 'yrsed'] = 15
df.loc[df['EDUC'] == 10, 'yrsed'] = 16
df.loc[df['EDUCD'] == 110, 'yrsed'] = 17
df.loc[df['EDUCD'] == 111, 'yrsed'] = 18
df.loc[df['EDUCD'] == 112, 'yrsed'] = 19
df.loc[df['EDUCD'] == 113, 'yrsed'] = 20
df.loc[df['EDUCD'] == 114, 'yrsed'] = 18
df.loc[df['EDUCD'] == 115, 'yrsed'] = 19
df.loc[df['EDUCD'] == 116, 'yrsed'] = 20

print(df)

      YEAR  SAMPLE   SERIAL       CBSERIAL   HHWT        CLUSTER   STRATA  GQ  \
0     2010  201001     7876         488424   90.0  2010000078761   240001   1   
1     2010  201001     7876         488424   90.0  2010000078761   240001   1   
2     2010  201001     7876         488424   90.0  2010000078761   240001   1   
3     2010  201001    22464        1388149  107.0  2010000224641   130001   1   
4     2010  201001    22464        1388149  107.0  2010000224641   130001   1   
...    ...     ...      ...            ...    ...            ...      ...  ..   
7033  2019  201901  1403102  2019000128243   37.0  2019014031021   130155   1   
7034  2019  201901  1404137  2019000186916   46.0  2019014041371   160055   1   
7035  2019  201901  1405932  2019000293128  123.0  2019014059321    90055   1   
7036  2019  201901  1413819  2019000743427  101.0  2019014138191  4100555   1   
7037  2019  201901  1418721  2019001023584  143.0  2019014187211    10255   1   

      PERNUM  PERWT  ...   

/tmp/ipykernel_67/1238924226.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['EDUCD'] == 21, 'yrsed'] = 5.5  # pre-2008


In [35]:
X = df[['yrsed']]
X = sm.add_constant(X)  
y = df['log_inctot']

model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             log_inctot   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.1714
Date:                Sat, 09 Mar 2024   Prob (F-statistic):              0.679
Time:                        08:08:02   Log-Likelihood:                -27911.
No. Observations:                7032   AIC:                         5.583e+04
Df Residuals:                    7030   BIC:                         5.584e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.1183      0.426     12.010      0.0

In [36]:
X = df[['RACE_Chinese', 'RACE_Indian', 'RACE_Other','AGE','SEX_2_Male','yrsed']]
X = sm.add_constant(X)  
y = df['log_inctot']

model = sm.OLS(y, X).fit(cov_type='HC3')

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             log_inctot   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     35.78
Date:                Sat, 09 Mar 2024   Prob (F-statistic):           6.61e-43
Time:                        08:08:07   Log-Likelihood:                -27805.
No. Observations:                7032   AIC:                         5.562e+04
Df Residuals:                    7025   BIC:                         5.567e+04
Df Model:                           6                                         
Covariance Type:                  HC3                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            1.4813      0.630      2.351   

In [22]:
print(df['CITIZEN'].value_counts())

CITIZEN
3    3540
2    3198
1     294
Name: count, dtype: int64


In [23]:
reg_df = df[df['CITIZEN'] != 1]

In [24]:
X = reg_df[['RACE_Chinese', 'RACE_Indian', 'RACE_Other','AGE','SEX_2_Male','yrsed']]
X = sm.add_constant(X)  
y = reg_df['log_inctot']

model = sm.OLS(y, X).fit(cov_type='HC3')

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             log_inctot   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.031
Method:                 Least Squares   F-statistic:                     39.91
Date:                Sat, 09 Mar 2024   Prob (F-statistic):           5.42e-48
Time:                        07:55:14   Log-Likelihood:                -26648.
No. Observations:                6738   AIC:                         5.331e+04
Df Residuals:                    6731   BIC:                         5.336e+04
Df Model:                           6                                         
Covariance Type:                  HC3                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            1.5323      0.573      2.673   

In [25]:
X = reg_df[['RACE_Chinese', 'RACE_Indian', 'RACE_Other']]
X = sm.add_constant(X)  
y = reg_df['log_inctot']

model2 = sm.OLS(y, X).fit(cov_type='HC3')

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             log_inctot   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.031
Method:                 Least Squares   F-statistic:                     39.91
Date:                Sat, 09 Mar 2024   Prob (F-statistic):           5.42e-48
Time:                        07:56:13   Log-Likelihood:                -26648.
No. Observations:                6738   AIC:                         5.331e+04
Df Residuals:                    6731   BIC:                         5.336e+04
Df Model:                           6                                         
Covariance Type:                  HC3                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            1.5323      0.573      2.673   

In [26]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

# Assuming you have your regression results stored in 'regression_result'

# Create a summary table
results_table = summary_col([model],   # List with a single regression result
                            float_format='%0.4f',  # Format for coefficients
                            stars=True,             # Display significance stars
                            model_names=['Your Model'],  # Model name
                            info_dict={'N': lambda x: "{0:d}".format(int(x.nobs)),  # Add custom information
                                       'R-squared': lambda x: "{:.4f}".format(x.rsquared)})

# Print the table
print(results_table)



               Your Model
-------------------------
const          1.5323*** 
               (0.5733)  
RACE_Chinese   0.9475**  
               (0.4373)  
RACE_Indian    1.0802    
               (0.6700)  
RACE_Other     3.2493*** 
               (0.5018)  
AGE            -0.0012   
               (0.0098)  
SEX_2_Male     3.7015*** 
               (0.3035)  
yrsed          0.1013*** 
               (0.0181)  
R-squared      0.0320    
R-squared Adj. 0.0312    
N              6738      
R-squared      0.0320    
Standard errors in
parentheses.
* p<.1, ** p<.05,
***p<.01


/srv/conda/lib/python3.9/site-packages/statsmodels/iolib/summary2.py:579: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dat = dat.applymap(lambda x: _formatter(x, float_format))


In [27]:
!pip install stargazer
import pandas as pd
from sklearn import datasets
import statsmodels.api as sm
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML

In [28]:
stargazer = Stargazer([model2,model])
HTML(stargazer.render_html())

In [33]:
column_summary = df['yrsed'].describe()
print(f"Summary for yrsed:\n{column_summary}")

Summary for yrsed:
count    7032.000000
mean        6.914036
std         7.860108
min         0.000000
25%         0.000000
50%         0.000000
75%        12.500000
max        20.000000
Name: yrsed, dtype: float64
